## **Feature Engineering for Crime Dataset**
* all_crime_features_2010_2023.csv (task2_eda_timeseries_analysis_byMihoRosenberg.ipynb) contains all original features and 8 more features created in the time series EDA: 
cleaned_occurence_hour	
    * week	
    * month	
    * year	
    * mon_year	
    * season	
    * is_holiday
    * is_weekend
* In this notebook, missing "NIBRS Class" values will be filled.
* "NIBRS Class" will be aggregated to its parent level (Ex, 13A, 13B, 13C --> 13)
* New feature will be added: 'Daily Crime Count' and 'Daily Crime Count by NIBRS'

In [1]:
#### Import the libraries needed
import pickle
import dill
import json
import glob
import os
from pathlib import Path
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy import stats

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
all_df = pd.read_csv('all_crime_features_2010_2023.csv', parse_dates=['Occurrence Date'], dtype={'Beat': str, 'ZIP Code': str, 'Offense Count': float, 'numeric': int})

In [3]:
all_df.head()

,Occurrence Date,Occurrence Hour,NIBRS Description,Offense Count,Beat,Premise,Block Range,Street Name,Street Type,Suffix,...,MapLongitude,MapLatitude,cleaned_occurence_hour,week,month,year,mon_year,season,is_holiday,is_weekend
0,2010-01-01,8.0,Theft,1.0,3B10,20R,4900-4999,POINCIANA,DR,-,...,NaN,NaN,8,Friday,1,2010,Jan-2010,Winter,1,0
1,2010-01-01,18.0,Theft,1.0,5F20,20D,8700-8799,HAMMERLY,-,-,...,NaN,NaN,18,Friday,1,2010,Jan-2010,Winter,1,0
2,2010-01-01,0.0,Burglary,1.0,1A10,05O,400-499,MAIN,ST,-,...,NaN,NaN,0,Friday,1,2010,Jan-2010,Winter,1,0
3,2010-01-01,0.0,Theft,1.0,7C10,20R,1900-1999,LOCKWOOD,DR,-,...,NaN,NaN,0,Friday,1,2010,Jan-2010,Winter,1,0
4,2010-01-01,10.0,Theft,1.0,18F20,18A,3300-3399,MCCUE,RD,-,...,NaN,NaN,10,Friday,1,2010,Jan-2010,Winter,1,0


In [4]:
all_df.shape

(2293309, 25)

### **Fill Missing Value in 'NIBRS Class'**
* Check unique combination of 'NIBRS Description' and 'NIBRS Class'
* Unique combination count: 137 
* Clean 'NIBRS Description'
    * Remove extra trailing spaces (Ex. 'Burglary      ')
    * Replace 'AutoTheft' with 'Auto Theft'
* Delete rows where 'NIBRS Description' is '1'
* Populate 'NIBRS Class' based on 'NIBRS Description'

|| Unique Value Count | Null Value Count |
|---|---|---|
| 'NIBRS Description' | 77 | 0 |
| 'NIBRS Class' | 62 | 74 |

In [5]:
print(f"Unique Count 'NIBRS Description': {all_df['NIBRS Description'].nunique()}")
print(f"Unique Count 'NIBRS Class': {all_df['NIBRS Class'].nunique()}")

Unique Count 'NIBRS Description': 77
Unique Count 'NIBRS Class': 62


In [6]:
# Extract unique combination of 'NIBRS Description' and 'NIBRS Class'
nibrs = all_df[['NIBRS Description', 'NIBRS Class']]
nibrs = nibrs.drop_duplicates()
print(f"Unique combination: {len(nibrs)}")

Unique combination: 137


In [7]:
# Unique 'NIBRS Description' and 'NIBRS Class' combination
class_fill = nibrs[~nibrs['NIBRS Class'].isnull()]['NIBRS Description']

print('----- NIBRS Description where NIBRS Class is filled -----')
print(f"Unique Value Count: {len(class_fill)}")
print(f"Unique 'NIBRS Description' Values:\n{sorted(class_fill.unique())}")

----- NIBRS Description where NIBRS Class is filled -----
Unique Value Count: 63
Unique 'NIBRS Description' Values:
['Aggravated Assault', 'All other larceny', 'All other offenses', 'Animal Cruelty', 'Arson', 'Assisting or promoting prostitution', 'Bad checks', 'Betting/wagering', 'Bribery', 'Burglary, Breaking and Entering', 'Counterfeiting, forgery', 'Credit card, ATM fraud', 'Curfew, loitering, vagrancy violations', 'Destruction, damage, vandalism', 'Disorderly conduct', 'Driving under the influence', 'Drug equipment violations', 'Drug, narcotic violations', 'Drunkenness', 'Embezzlement', 'Extortion, Blackmail', 'False pretenses, swindle', 'Family offenses, no violence', 'Forcible fondling', 'Forcible rape', 'Forcible sodomy', 'From coin-operated machine or device', 'Gambling equipment violations', 'Hacking/Computer Invasion', 'Human Trafficking/Commercial Sex Act', 'Human Trafficking/Involuntary Servitude', 'Identify theft', 'Impersonation', 'Incest', 'Intimidation', 'Justifiable h

In [8]:
# Check null in 'NIBRS Class'
class_null = nibrs[nibrs['NIBRS Class'].isnull()]['NIBRS Description']

print('----- NIBRS Description where NIBRS Class is null -----')
print(f"Unique Value Count: {len(class_null)}")
print(f"Unique 'NIBRS Description' Values:\n{sorted(class_null.unique())}")

----- NIBRS Description where NIBRS Class is null -----
Unique Value Count: 74
Unique 'NIBRS Description' Values:
['1', 'Aggravated Assault', 'Aggravated Assault       ', 'All other larceny', 'All other offenses', 'Animal Cruelty', 'Arson', 'Assisting or promoting prostitution', 'Auto Theft', 'AutoTheft', 'AutoTheft                ', 'Bad checks', 'Betting/wagering', 'Bribery', 'Burglary', 'Burglary                 ', 'Burglary, Breaking and Entering', 'Counterfeiting, forgery', 'Credit card, ATM fraud', 'Curfew, loitering, vagrancy violations', 'Destruction, damage, vandalism', 'Disorderly conduct', 'Driving under the influence', 'Drug equipment violations', 'Drug, narcotic violations', 'Drunkenness', 'Embezzlement', 'Extortion, Blackmail', 'False pretenses, swindle', 'Family offenses, no violence', 'Forcible fondling', 'Forcible rape', 'Forcible sodomy', 'From coin-operated machine or device', 'Gambling equipment violations', 'Hacking/Computer Invasion', 'Human Trafficking/Commercial

In [9]:
# Check values in 'NIBRS Description' where 'NIBRS Class' has null
set_null = set(class_null)
set_fill = set(class_fill)

# "NIBRS Description": Contained only in set_null
set_null.difference(set_fill)

{'1',
 'Aggravated Assault       ',
 'Auto Theft',
 'AutoTheft',
 'AutoTheft                ',
 'Burglary',
 'Burglary                 ',
 'Murder',
 'Murder                   ',
 'Rape',
 'Rape                     ',
 'Robbery                  ',
 'Theft',
 'Theft                    '}

### **Clean 'NIBRS Description'**
* Remove extra spaces 
* Replace 'AutoTheft' with 'Auto Theft'
* Check null in 'NIBRS Class' again

In [10]:
# Change 'NIBRS Description' to lower case
replaced_description = [dsc.lower() for dsc in all_df['NIBRS Description']]
# Replace 'AutoTheft' with 'Auto Theft'
replaced_description = [dsc.replace('autotheft', 'auto theft') for dsc in replaced_description]

# Delete extra spaces
pattern = "[\s]+$"
cleaned_description = [re.sub("[\s]+$", "", s) for s in replaced_description]

In [11]:
# Add 'cleaned_description' to all_df
all_df['cleaned_description'] = cleaned_description

In [12]:
all_df['cleaned_description'].unique()

array(['theft', 'burglary', 'robbery', 'auto theft', 'aggravated assault',
       'rape', 'murder', '1', 'simple assault', 'all other offenses',
       'destruction, damage, vandalism', 'shoplifting',
       'motor vehicle theft', 'theft from building',
       'theft from motor vehicle', 'burglary, breaking and entering',
       'false pretenses, swindle', 'intimidation', 'purse-snatching',
       'all other larceny', 'theft of motor vehicle parts or accessory',
       'weapon law violations', 'drug, narcotic violations',
       'trespass of real property', 'pocket-picking',
       'counterfeiting, forgery', 'identify theft',
       'credit card, atm fraud', 'animal cruelty',
       'family offenses, no violence', 'driving under the influence',
       'impersonation', 'bad checks', 'statutory rape', 'forcible rape',
       'disorderly conduct', 'drug equipment violations',
       'stolen property offenses', 'drunkenness', 'prostitution',
       'extortion, blackmail', 'human traffickin

In [13]:
# Check 'cleaned_description', and 'NIBRS Class' again
nibrs_cleaned_desc = all_df[['cleaned_description', 'NIBRS Class']]
nibrs_cleaned_desc = nibrs_cleaned_desc.drop_duplicates()

In [14]:
len(nibrs_cleaned_desc)

128

In [15]:
# Extract rows where 'NIBRS Class' is NOT null
nibrs_cleaned_class_fill = nibrs_cleaned_desc[~nibrs_cleaned_desc['NIBRS Class'].isnull()]

# Convert dataframe to dictionalry
dict_desc_class = nibrs_cleaned_class_fill.set_index('cleaned_description')['NIBRS Class'].to_dict()

In [16]:
dict_desc_class

{'destruction, damage, vandalism': '290',
 'drug, narcotic violations': '35A',
 'weapon law violations': '520',
 'aggravated assault': '13A',
 'all other larceny': '23H',
 'simple assault': '13B',
 'drug equipment violations': '35B',
 'robbery': '120',
 'intimidation': '13C',
 'false pretenses, swindle': '26A',
 'theft from motor vehicle': '23F',
 'credit card, atm fraud': '26B',
 'burglary, breaking and entering': '220',
 'motor vehicle theft': '240',
 'wire fraud': '26E',
 'pornographs, obscene material': '370',
 'theft from building': '23D',
 'identify theft': '26F',
 'forcible sodomy': '11B',
 'forcible rape': '11A',
 'human trafficking/commercial sex act': '64A',
 'counterfeiting, forgery': '250',
 'shoplifting': '23C',
 'forcible fondling': '11D',
 'statutory rape': '36B',
 'embezzlement': '270',
 'purse-snatching': '23B',
 'murder, non-negligent': '09A',
 'theft of motor vehicle parts or accessory': '23G',
 'assisting or promoting prostitution': '40B',
 'impersonation': '26C',
 

In [17]:
# Extract rows where 'NIBRS Class' is null
nibrs_cleaned_class_null = nibrs_cleaned_desc[nibrs_cleaned_desc['NIBRS Class'].isnull()]

In [18]:
# Check 'NIBRS Class' where 'cleaned_description' is '1'
null_ratio = len(all_df[all_df['cleaned_description'] == '1']['NIBRS Class']) / all_df[all_df['cleaned_description'] == '1']['NIBRS Class'].isna().sum()
print(f"Null ratio: {null_ratio*100}%")

Null ratio: 100.0%


In [19]:
# Check 'cleaned_description' where 'NIBRS Class' is null again
set_nibrs_cleaned_class_fill = set(nibrs_cleaned_class_fill['cleaned_description'])
set_nibrs_cleaned_class_null = set(nibrs_cleaned_class_null['cleaned_description'])

# "NIBRS Description": Contained only in set_nibrs_cleaned_class_null
set_nibrs_cleaned_class_null.difference(set_nibrs_cleaned_class_fill) # {'1', 'auto theft', 'burglary', 'murder', 'rape', 'theft'}

{'1', 'auto theft', 'burglary', 'murder', 'rape', 'theft'}

In [20]:
print(len(all_df))
print(len(all_df[all_df['cleaned_description'] == '1'])) # 14

2293309
14


In [21]:
# Delete rows where 'cleaned_description' is '1'
all_df = all_df[~(all_df['cleaned_description'] == '1')].reset_index()

In [22]:
# Replace 'auto theft' with "motor vehicle theft" and set 'NIBRS Class' to 240
# Replace 'burglary' with "burglary, breaking and entering" and set 'NIBRS Class' to 220
# Replace 'murder' with "Murder, non-negligent" to 09A
# Replace 'rape' with "forcible rape" to 11A
# Replace 'theft' with "all other larceny" to 23H

desc_map = {'auto theft': "motor vehicle theft", 
            'burglary': "burglary, breaking and entering", 
            'murder': "murder, non-negligent", 
            'rape': "forcible rape", 
            'theft': "all other larceny"}

all_df['cleaned_description'].replace(desc_map, inplace=True)

In [23]:
# Check these values are gone
all_df['cleaned_description'].isin(['1', 'auto theft', 'burglary', 'murder', 'rape', 'theft']).sum() # 0

0

In [24]:
# Manually add these to dict_desc_class {'1', 'auto theft', 'burglary', 'murder', 'rape', 'theft'}
class_map = {"motor vehicle theft": '240', 
             "burglary, breaking and entering": '220', 
             "murder, non-negligent": '09A', 
             "forcible rape": '11A', 
             "all other larceny": '23H'}

# Add class_map to dict_desc_class
for k, v in class_map.items():
    dict_desc_class[k] = v

In [25]:
# Replace a null value with a value based on 'cleaned_description'
cleaned_class = [dict_desc_class[dsc] for dsc in all_df['cleaned_description']]  
all_df['cleaned_class'] = cleaned_class      

### **Verify 'cleaned_class' is populated**
* According to "NIBRS Offense Codes (NIBRS_Offense_Codes.pdf)*, there are 22 Group A offense categories making up 46 Group A offenses and  11 Group B offense categories.". Thus, the number of unique combination should be less than or equal to 57.  
* '26F', '26G', '40C', '64A', '64B', '720' are included in 'NIBRS Class', which are not listed in "NIBRS Offense Codes".
* Extract 'cleaned_class' and 'cleaned_description' and check:
    * Null exists
    * Number of unique combination

###### * https://www.houstontx.gov/police/cs/Monthly_Crime_Data_by_Street_and_Police_Beat.htm

In [26]:
# Extract unique combination of 'cleaned_class' and 'cleaned_description'
df_to_verify = all_df[['cleaned_class', 'cleaned_description']]

In [27]:
print(f"----- Number of unique value -----:\n{df_to_verify.nunique()}")
df_to_verify.value_counts(['cleaned_class', 'cleaned_description']).sort_values()

----- Number of unique value -----:
cleaned_class          62
cleaned_description    62
dtype: int64


cleaned_class  cleaned_description                    
36A            incest                                          5
09C            justifiable homicide                           19
64B            human trafficking/involuntary servitude        19
90I            runaway                                        23
39A            betting/wagering                               25
                                                           ...  
23F            theft from motor vehicle                   145349
13A            aggravated assault                         162035
240            motor vehicle theft                        183151
220            burglary, breaking and entering            263686
23H            all other larceny                          663850
Name: count, Length: 62, dtype: int64

In [28]:
df_to_verify.drop_duplicates(inplace=True)

In [29]:
len(df_to_verify)

62

In [30]:
cleaned_class_vals = all_df['cleaned_class'].unique()
sorted = sorted(cleaned_class_vals)

In [31]:
# Check each value --> Found '26F', '26G', '40C', '64A', '64B', '720' are in "cleaned_class"
print(sorted[:25])
print(sorted[25:50])
print(sorted[50:])

['09A', '09B', '09C', '100', '11A', '11B', '11C', '11D', '120', '13A', '13B', '13C', '200', '210', '220', '23A', '23B', '23C', '23D', '23E', '23F', '23G', '23H', '240', '250']
['26A', '26B', '26C', '26D', '26E', '26F', '26G', '270', '280', '290', '35A', '35B', '36A', '36B', '370', '39A', '39B', '39C', '40A', '40B', '40C', '510', '520', '64A', '64B']
['720', '90A', '90B', '90C', '90D', '90E', '90F', '90G', '90H', '90I', '90J', '90Z']


In [32]:
# Check if there is null in 'cleaned_class'
df_to_verify_class_nan = df_to_verify[df_to_verify['cleaned_class'].isna()]['cleaned_description']
print(len(df_to_verify_class_nan)) # 0

0


In [33]:
# Check not listed 'cleaned_description' and 'cleaned_class'
not_listed_nibrs_class = all_df[all_df['cleaned_class'].isin(['26F', '26G', '40C', '64A', '64B', '720'])]
not_listed_nibrs_class[['cleaned_description', 'cleaned_class']].drop_duplicates()

,cleaned_description,cleaned_class
1036975,identify theft,26F
1037031,animal cruelty,720
1037313,human trafficking/commercial sex act,64A
1037567,hacking/computer invasion,26G
1041179,purchasing prostitution,40C
1079367,human trafficking/involuntary servitude,64B


In [34]:
# All null values are populated
all_df['cleaned_class'].isna().sum()

0

### **Aggregate 'cleaned_class'**
* Aggregate 'cleaned_class' to its parent level (Ex, 13A, 13B, 13C --> 13)

In [43]:
aggregate_cleaned_class = []
for code in all_df['cleaned_class']:
    parent = ""
    for i in range(len(code)):
        if code[i].isnumeric():
            parent += code[i]
    if parent == '90':
        parent = code
    aggregate_cleaned_class.append(parent) 

In [44]:
# Check 'aggregate_cleaned_class'
aggregate_cleaned_class 

['23',
 '23',
 '220',
 '23',
 '23',
 '23',
 '23',
 '220',
 '220',
 '23',
 '23',
 '120',
 '120',
 '23',
 '240',
 '220',
 '23',
 '23',
 '23',
 '23',
 '220',
 '23',
 '23',
 '23',
 '23',
 '23',
 '220',
 '23',
 '23',
 '23',
 '13',
 '220',
 '220',
 '23',
 '23',
 '240',
 '220',
 '13',
 '220',
 '120',
 '120',
 '23',
 '23',
 '23',
 '23',
 '220',
 '240',
 '240',
 '13',
 '13',
 '23',
 '11',
 '23',
 '23',
 '13',
 '13',
 '220',
 '220',
 '23',
 '23',
 '13',
 '23',
 '23',
 '240',
 '220',
 '240',
 '23',
 '23',
 '23',
 '13',
 '23',
 '220',
 '120',
 '23',
 '23',
 '13',
 '220',
 '220',
 '220',
 '220',
 '23',
 '23',
 '23',
 '23',
 '23',
 '120',
 '23',
 '23',
 '240',
 '220',
 '23',
 '220',
 '220',
 '23',
 '23',
 '23',
 '220',
 '240',
 '13',
 '120',
 '23',
 '13',
 '220',
 '220',
 '23',
 '23',
 '23',
 '13',
 '23',
 '23',
 '23',
 '23',
 '23',
 '13',
 '23',
 '23',
 '23',
 '23',
 '220',
 '23',
 '23',
 '23',
 '23',
 '23',
 '220',
 '13',
 '13',
 '120',
 '120',
 '13',
 '13',
 '13',
 '23',
 '13',
 '220',
 '13',
 '2

In [45]:
# Check unique values in 'aggregate_cleaned_class'
set(aggregate_cleaned_class)

{'09',
 '100',
 '11',
 '120',
 '13',
 '200',
 '210',
 '220',
 '23',
 '240',
 '250',
 '26',
 '270',
 '280',
 '290',
 '35',
 '36',
 '370',
 '39',
 '40',
 '510',
 '520',
 '64',
 '720',
 '90A',
 '90B',
 '90C',
 '90D',
 '90E',
 '90F',
 '90G',
 '90H',
 '90I',
 '90J',
 '90Z'}

In [46]:
# Add 'aggregate_cleaned_class' to all_df
all_df['aggregate_cleaned_class'] = aggregate_cleaned_class

In [39]:
# Check all_df
all_df.head()

,index,Occurrence Date,Occurrence Hour,NIBRS Description,Offense Count,Beat,Premise,Block Range,Street Name,Street Type,...,week,month,year,mon_year,season,is_holiday,is_weekend,cleaned_description,cleaned_class,aggregate_cleaned_class
0,0,2010-01-01,8.0,Theft,1.0,3B10,20R,4900-4999,POINCIANA,DR,...,Friday,1,2010,Jan-2010,Winter,1,0,all other larceny,23H,23
1,1,2010-01-01,18.0,Theft,1.0,5F20,20D,8700-8799,HAMMERLY,-,...,Friday,1,2010,Jan-2010,Winter,1,0,all other larceny,23H,23
2,2,2010-01-01,0.0,Burglary,1.0,1A10,05O,400-499,MAIN,ST,...,Friday,1,2010,Jan-2010,Winter,1,0,"burglary, breaking and entering",220,220
3,3,2010-01-01,0.0,Theft,1.0,7C10,20R,1900-1999,LOCKWOOD,DR,...,Friday,1,2010,Jan-2010,Winter,1,0,all other larceny,23H,23
4,4,2010-01-01,10.0,Theft,1.0,18F20,18A,3300-3399,MCCUE,RD,...,Friday,1,2010,Jan-2010,Winter,1,0,all other larceny,23H,23


### **Drop 'Occurrence Hour', 'NRBIS Description', and 'NRBIS Class'**
* 'Occurrence Hour' --> 'cleaned_occurence_hour'
* 'NIBRS Description' --> 'cleaned_description'
* 'NIBRS Class' --> 'cleaned_class'

In [50]:
all_df.drop(['Occurrence Hour', 'NIBRS Description', 'NIBRS Class'], axis=1, inplace=True)

### **Save all_df to**

In [56]:
# Save all_df to all_crime_features_2010_2023.csv
all_df.to_csv('all_crime_features_2010_2023_w_nibrs_class.csv', index=False)